In [13]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import ChatPromptTemplate


import sys
sys.path.append('../../')  # replace with the actual path
import lib.key_param as key_param
import lib.Utils as utils

In [14]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = key_param.LANGSMITH_API_KEY
os.environ['OPENAI_API_KEY'] = key_param.OPENAI_API_KEY

In [15]:
# %%capture # Avoid printing the output

# Get a list of all .pdf files in the directory
pdf_files = [f for f in os.listdir("../../sample_files") if f.endswith('.pdf')]

pages = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(f"../../sample_files/{pdf_file}")
    pages.extend(loader.load_and_split())
    
docs = pages

Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEncoding not implemented yet
Advanced encoding /SymbolSetEnc

In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [17]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [18]:
# Prompt
template_rag = """System Message:

You are a privacy consultant for a marketing company that operate internationally. Provide your legal advice based on the context.

Context: {context}

Question: {question}
"""

prompt_rag = ChatPromptTemplate.from_template(template_rag)


In [19]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [20]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [21]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [22]:
output_rag = rag_chain.invoke("In the context of the material scope of application of GDPR, what is the edpb position in relation to the one-stop shop mechanism?")

In [23]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template_no_rag = """Question: {question}
"""

prompt_no_rag = ChatPromptTemplate.from_template(template_no_rag)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain_no_rag = prompt_no_rag | llm | StrOutputParser()

output_no_rag = chain_no_rag.invoke({"question":"In the context of the material scope of application of GDPR, what is the edpb position in relation to the one-stop shop mechanism?"})

In [24]:
prompt_no_rag.pretty_print()

================================ Human Message =================================

Question: {question}


In [25]:
print("--- RAG ---\n")
print(utils.format_print(output_rag, 100))

--- RAG ---

The EDPB confirms that in the absence of an establishment in the Union, a controller or processor
cannot benefit from the one-stop shop mechanism provided for in Article 56 of the GDPR. The GDPR’s
cooperation and consistency mechanism only applies to controllers and processors with an
establishment, or establishments, within the European Union. Controllers and processors will need to
take into account other applicable texts, such as EU or Member States’ sectorial legislation and
national laws, in addition to the GDPR.


In [28]:
output_no_rag = llm.invoke("question In the context of the material scope of application of GDPR, what is the edpb position in relation to the one-stop shop mechanism?")

print("--- No RAG ---\n", output_no_rag)

--- No RAG ---
 content='The European Data Protection Board (EDPB) has stated that the one-stop shop mechanism provided for in the GDPR allows for a more efficient and consistent application of data protection rules across the EU. This mechanism allows businesses operating in multiple EU countries to deal with a single lead supervisory authority for their data protection issues, rather than having to deal with multiple authorities in each country where they operate. The EDPB has emphasized the importance of cooperation between supervisory authorities in ensuring the effective implementation of the one-stop shop mechanism and has provided guidance on how this mechanism should be applied in practice.' response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 35, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-683ea948-2e01-438a-88be-558c8be4a424-0'
